# Lesson 3: Connecting to a SQL Database

## Setup

In [1]:
## import os
from IPython.display import Markdown, HTML, display
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI
from langchain_groq import ChatGroq

## Recover the original dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [2]:
ls

 Volume in drive D is Data
 Volume Serial Number is B8DC-02F7

 Directory of D:\AI_team\SQL_Groq

17-06-2024  23:10    <DIR>          .
17-06-2024  23:03    <DIR>          ..
17-06-2024  23:04    <DIR>          .ipynb_checkpoints
17-06-2024  22:36    <DIR>          __pycache__
17-06-2024  22:36             5,247 app.py
17-06-2024  21:04             5,144 app1.py
17-06-2024  23:10            36,643 L3_Connecting_to_a_SQL_Database.ipynb
17-06-2024  21:09             4,955 main.py
17-06-2024  22:57             3,955 main2.py
16-06-2024  12:10           170,878 PO.csv
16-06-2024  09:36             5,150 sql_agent.ipynb
17-06-2024  23:09           204,800 test.db
               8 File(s)        436,772 bytes
               4 Dir(s)  142,707,228,672 bytes free


In [3]:
from sqlalchemy import create_engine
import pandas as pd


df = pd.read_csv("PO.csv").fillna(value = 0)

## Move the data to the SQL database

In [4]:
# Path to your SQLite database file
database_file_path = "test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
file_url = "PO.csv"
df = pd.read_csv(file_url).fillna(value = 0)
df.to_sql(
    'PO',
    con=engine,
    if_exists='replace',
    index=False
)

202

## Prepare the SQL prompt

In [5]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [6]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

## Call the Azure Chat model and create the SQL agent

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [7]:
"""llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-1106",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    temperature=0, 
    max_tokens=500
)"""

llm = ChatGroq(
      temperature=0, 
      groq_api_key = "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", 
      model_name="llama3-70b-8192"
        )

db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
QUESTION = input("Qestion:?")

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

agent_executor_SQL.invoke(QUESTION)

Qestion:? what is the spend in Event Management                                                                                      Explanation:\nI first listed the tables in the database using `sql_db_list_tables` and found the `PO` and `all_states_history` tables. Then, I used `sql_db_schema` to get the schema of these tables. I found that the `PO` table has a `Category` column and a `Subcategory` column, and I suspected that the correct category for Event Management is `'Management'` with `'Event Management'` as the `Subcategory`. I used `sql_db_query_checker` to check the query and then executed it using `sql_db_query` to get the sum of `TotalCost` for the records with the correct category and subcategory.




> Entering new AgentExecutor chain...
Thought: I need to find the spend in Event Management, but I don't know which table to query. 

Action: sql_db_list_tables
Action Input: POAction: sql_db_schema
Action Input: PO
CREATE TABLE "PO" (
	"PONumber" TEXT, 
	"Catalog ID" TEXT, 
	"ItemID" TEXT, 
	"Item Name" TEXT, 
	"Form" TEXT, 
	"When to use the form" TEXT, 
	"Description" TEXT, 
	"Specifications" TEXT, 
	"Category" TEXT, 
	"Subcategory" TEXT, 
	"UserID" TEXT, 
	"User Name" TEXT, 
	"SupplierID" TEXT, 
	"Country" TEXT, 
	"Contact Number" TEXT, 
	"Email" TEXT, 
	"PurchaseDate" TEXT, 
	"DeliveryDate" TEXT, 
	"Quantity" BIGINT, 
	"Unit Price" FLOAT, 
	"TotalCost" FLOAT, 
	"CostCenter" TEXT, 
	"GLAccount" TEXT, 
	"Currency" TEXT, 
	"Budget Owner" TEXT, 
	"Budget" BIGINT, 
	"Budget Available" FLOAT, 
	"Department" TEXT, 
	"Role" TEXT, 
	"Rating" FLOAT, 
	"Avg Delivery Time (Days)" BIGINT, 
	"Lead Time(hrs)" BIGINT, 
	"Reliability Score" FLOAT, 
	"Quarter" TEXT, 
	"Spend Type" TEXT, 
	"Suppli

In [9]:
QUESTION = input("Qestion:?")

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

agent_executor_SQL.invoke(QUESTION)

Qestion:? can give me a break up of top 5 suppliers & total PO spend of each supplier




> Entering new AgentExecutor chain...
Thought: I need to know the tables and columns available in the database to answer this question.

Action: sql_db_list_tables
Action Input: PO, all_states_historyAction: sql_db_schema
Action Input: PO, all_states_history
CREATE TABLE "PO" (
	"PONumber" TEXT, 
	"Catalog ID" TEXT, 
	"ItemID" TEXT, 
	"Item Name" TEXT, 
	"Form" TEXT, 
	"When to use the form" TEXT, 
	"Description" TEXT, 
	"Specifications" TEXT, 
	"Category" TEXT, 
	"Subcategory" TEXT, 
	"UserID" TEXT, 
	"User Name" TEXT, 
	"SupplierID" TEXT, 
	"Country" TEXT, 
	"Contact Number" TEXT, 
	"Email" TEXT, 
	"PurchaseDate" TEXT, 
	"DeliveryDate" TEXT, 
	"Quantity" BIGINT, 
	"Unit Price" FLOAT, 
	"TotalCost" FLOAT, 
	"CostCenter" TEXT, 
	"GLAccount" TEXT, 
	"Currency" TEXT, 
	"Budget Owner" TEXT, 
	"Budget" BIGINT, 
	"Budget Available" FLOAT, 
	"Department" TEXT, 
	"Role" TEXT, 
	"Rating" FLOAT, 
	"Avg Delivery Time (Days)" BIGINT, 
	"Lead Time(hrs)" BIGINT, 
	"Reliability Score" FLOAT, 
	"Qu

{'input': 'can give me a break up of top 5 suppliers & total PO spend of each supplier',
 'output': 'The top 5 suppliers and their total PO spend are: \n1. FutureTech Enterprises - 350000.0\n2. Data Dynamics Corp. - 316000.0\n3. Global Services Co. - 196000.0\n4. Tech Solutions Inc. - 173000.0\n5. Innovative IT Ltd. - 169000.0'}

In [10]:
QUESTION = input("Qestion:?")

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

agent_executor_SQL.invoke(QUESTION)

Qestion:? Which supplier is highly rated?




> Entering new AgentExecutor chain...
Thought: I need to know the table names in the database to start with.

Action: sql_db_list_tables
Action Input: PO, all_states_historyAction: sql_db_schema
Action Input: PO, all_states_history
CREATE TABLE "PO" (
	"PONumber" TEXT, 
	"Catalog ID" TEXT, 
	"ItemID" TEXT, 
	"Item Name" TEXT, 
	"Form" TEXT, 
	"When to use the form" TEXT, 
	"Description" TEXT, 
	"Specifications" TEXT, 
	"Category" TEXT, 
	"Subcategory" TEXT, 
	"UserID" TEXT, 
	"User Name" TEXT, 
	"SupplierID" TEXT, 
	"Country" TEXT, 
	"Contact Number" TEXT, 
	"Email" TEXT, 
	"PurchaseDate" TEXT, 
	"DeliveryDate" TEXT, 
	"Quantity" BIGINT, 
	"Unit Price" FLOAT, 
	"TotalCost" FLOAT, 
	"CostCenter" TEXT, 
	"GLAccount" TEXT, 
	"Currency" TEXT, 
	"Budget Owner" TEXT, 
	"Budget" BIGINT, 
	"Budget Available" FLOAT, 
	"Department" TEXT, 
	"Role" TEXT, 
	"Rating" FLOAT, 
	"Avg Delivery Time (Days)" BIGINT, 
	"Lead Time(hrs)" BIGINT, 
	"Reliability Score" FLOAT, 
	"Quarter" TEXT, 
	"Spend Type"

{'input': 'Which supplier is highly rated?',
 'output': 'The highly rated supplier is Martinez, Gordon and Allen with an average rating of 4.81.'}